In [ ]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import re

from datetime import datetime 
from collections import defaultdict
from scipy.stats import pearsonr

In [ ]:
users = pd.read_csv('new_users.csv')

### Summing parameters in users (likes received, retweet and reply received)

In [ ]:
new_users = users.merge(tweets.groupby('user_id').sum(), on='user_id')
new_users = new_users.drop(columns=\
    ['infer_urls_x', 'infer_hashtags_y',	'infer_mentions_y',\
    'infer_urls_y', 'infer_hashtags_x', 'infer_mentions_x', 'id']\
        )
new_users = new_users.rename(columns={
    'retweet_count' : 'retweet_received',
    'reply_count' : 'reply_received',
    'favorite_count' : 'favorite_received',
    'num_hashtags' : 'hashtag_used',
    'num_urls' : 'urls_used',
    'num_mentions' : 'mentions_used'}, 
    errors='raise'
)
new_users.head()

In [ ]:
new_users[\
    (new_users['retweet_received'].isna() == True) |\
    (new_users['reply_received'].isna() == True) |\
    (new_users['favorite_received'].isna() == True) |\
    (new_users['hashtag_used'].isna() == True) |\
    (new_users['urls_used'].isna() == True) |\
    (new_users['mentions_used'].isna() == True)
        ].head()

In [ ]:
new_users['reply_received'] = new_users['reply_received'].fillna(-1)
new_users['retweet_received'] = new_users['retweet_received'].fillna(-1)
new_users['statuses_count'] = new_users['statuses_count'].fillna(-1)

In [ ]:
new_users['reply_received'] = new_users['reply_received'].astype(int)
new_users['retweet_received'] = new_users['retweet_received'].astype(int)
new_users['favorite_received'] = new_users['favorite_received'].astype(int)
new_users['hashtag_used'] = new_users['hashtag_used'].astype(int)
new_users['urls_used'] = new_users['urls_used'].astype(int)
new_users['mentions_used'] = new_users['mentions_used'].astype(int)
new_users['statuses_count'] = new_users['statuses_count'].astype(int)

### Ratio

In [ ]:
new_users['like/statuses'] = round(new_users['favorite_received'] / new_users['count'], 2)
new_users.head(3)

## Number of special char

In [ ]:
special = '[(|!£$%&/=?^@#§,.;:-_<>ç@)]' # Define special characters
tweets['special'] = tweets['text'].str.count(special) # Count them
tweets['special'] = tweets['special'].fillna(0)
tweets['special'] = tweets['special'].astype(int)


## Lenght

In [ ]:
tweets['text_lenght'] = tweets['text'].str.len()
tweets['text_lenght'] = tweets['text_lenght'].fillna(0)
tweets['text_lenght'] = tweets['text_lenght'].astype(int)


#### Merging

In [ ]:
mean_tweets = tweets.groupby('user_id').mean(numeric_only=True)


In [ ]:
mean_tweets = mean_tweets.drop(columns=['retweet_count', 'reply_count',\
    	'favorite_count', 'num_hashtags',\
        'num_urls',	'num_mentions',\
        'infer_hashtags_x',	'infer_mentions_x', 'infer_urls_x', 'infer_hashtags_y',\
        'infer_mentions_y', 'infer_urls_y', 'id'\
        ])



In [ ]:

new_users = new_users.merge(mean_tweets, on='user_id')
new_users = new_users.rename(columns={'text_lenght':'avg_text_len', 'special':'avg_special'})


In [ ]:
new_users.head(5)

In [ ]:

new_users['avg_special'] = round(new_users['avg_special'], 2)
new_users['avg_text_len'] = round(new_users['avg_text_len'], 2)

In [ ]:
new_users.head(5)

In [ ]:
users = new_users

In [ ]:
users[users['like/statuses'] == users['like/statuses'].max()]

In [ ]:
plt.hist(users['like/statuses'], bins=round(math.log2(users['like/statuses'].max()) + 1), log=True)
plt.show()